# Material Input

In [ ]:
fck=30 #MPa
fyk=500 #MPa
fywk=500 #MPa
max_util=0.9
a_cover = 35

Importing libraries

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from EC2_Beam_v1 import *
import openpyxl as opxl
from itertools import zip_longest
import re

# Read 1D member Geometry Data

In [ ]:
Member=pd.read_excel("Member.xlsx")
Rows = len(Member)
Member.dropna(inplace=True)
a=Member["Name"]
print(len(a))

prop=pd.read_excel("Property.xlsx")
prop.dropna()
prop.info()
Des=[x.split(" ") for x in prop.Description]
Profile=[]
for x in Des:
    Shape=x[1][0]
    u=1000 if "(m)" in x[1] else 1
    if Shape=="R":
        D=float(x[2])*u
        B=float(x[3])*u
    elif Shape=="C":
        D=float(x[2])*u
        B=0
    else:
        D=0
        B=0
    Profile.append([Shape,D,B])
prop["Shape"],prop["D"],prop["B"]=zip(*Profile)


force=pd.read_csv("MembForce.csv",skiprows = 28,usecols=["Memb","Pos","Case","Fx","Fy","Fz","Mxx","Myy","Mzz"])
force.rename(columns={"Memb":"ID"},inplace = True)
force.dropna(inplace=True)
force = force.astype({"ID": int})
force["Pos"]= [float(x.strip('%'))/100 for x in force["Pos"]]

In [112]:
Member=pd.merge(Member,prop,how="left",on="Property",suffixes=('', '_'))
Member = Member.astype({"ID": int})

force=pd.read_csv("MembForce.csv",skiprows = 28,usecols=["Memb","Pos","Case","Fx","Fy","Fz","Mxx","Myy","Mzz"])
force.rename(columns={"Memb":"ID"},inplace = True)
force.dropna(inplace=True)
force = force.astype({"ID": int})
force["Pos"]= [float(x.strip('%'))/100 for x in force["Pos"]]

Beam=pd.merge(Member[["Name","ID","Length","D","B"]],force,how='left',on='ID')
Beam["seg"]=force.Pos.apply(lambda x: "Left" if x <= 0.25 else "Mid" if x<=0.75 else "Right")
Beam.dropna(inplace=True)
Beam=Beam.sort_values(by=['Name','Pos'])

beam_list=pd.unique(Beam['Name']).tolist()
beam_number = len(beam_list)
span_type = []
for i in range(0,beam_number-1):
    if 'a' in beam_list[i]:
        result = "1st"
    elif len(str(beam_list[i])) == 6:
        result = "single"
    elif (len(str(beam_list[i+1])) == 6) or 'a' in beam_list[i+1]:
        result = "end"
    else:
        result = "int"
    span_type.append(result)
span_type.append("end")
span=pd.DataFrame()
span['Span type']=span_type
span['Name']=beam_list
Beam=pd.merge(Beam,span,how="left",on="Name",suffixes=('', '_'))

print(beam_number)
Beam.to_csv('Beam.csv')

229


Rebar calculation

In [ ]:
As_cal=[As_rq(fck,fyk,M,D,B) for M,D,B in zip(Beam.Myy,Beam.D,Beam.B)]
Result=Beam.loc[:,["Name","Span type","ID","seg",'Length','B','D',"Myy",'Fz']]
Result["As"], Result["Asc"], Result["As_top"], Result["As_bot"],Result["k"],Result["z"]=  zip(*As_cal)
Result.head(20)

Shear calculation
Shear_status=0 : OK
Shear_status=1 : Fail

In [ ]:
shear_cal=[Shear_cal(fck,fywk,V_Ed,D,B) for V_Ed,D,B in zip(Beam.Fz,Beam.D,Beam.B)]
Result["Asw_s"],Result["v_Ed"],Result["cot_theta"],Result["Shear_status"]=zip(*shear_cal)
Result["Asw_s"] = Result["Asw_s"].astype(float)
Result.head(20)

In [ ]:
Rebar=Result[["Name","Span type","ID","seg","Length",'B','D',"As_top","As_bot","Asw_s",'Shear_status']].groupby(["Name","ID","seg"]).max()
Rebar["As_tol"]=Rebar.As_top + Rebar.As_bot
Rebar["p_top"]=Rebar.As_top/Rebar.B/Rebar.D
Rebar["p_bot"]=Rebar.As_bot/Rebar.B/Rebar.D
Rebar["p_tol"]=Rebar.As_tol/Rebar.B/Rebar.D
Rebar["Bend_status"]=Rebar.p_tol.apply(lambda x: 0 if x <= 0.04 else 1)
Rebar=Rebar[["Span type","Length",'B','D',"As_top","As_bot","p_top","p_bot","p_tol","Asw_s","Bend_status",'Shear_status']]
Rebar.loc[(Rebar.Shear_status == 1) | (Rebar.Bend_status == 1) ]
Rebar
#Rebar.isna()
# Rebar.to_csv('Output.csv')

# Beam Maximum Steel Ratio Diagram
Maxium Ratio of each beam

In [ ]:
ratio_max=Rebar.groupby(["ID"]).max()["p_tol"]
#ratio_max.plot.hist()
sns.distplot(ratio_max)
plt.axvline(x=np.mean(ratio_max),c='red',ls='--',label='mean')
plt.axvline(x=np.percentile(ratio_max,25),c='green',ls='--',label='25-75th percentile')
plt.axvline(x=np.percentile(ratio_max,75),c='green',ls='--')
plt.legend()

In [ ]:
Asw_s_max=Rebar.groupby(["ID"]).max()["Asw_s"]
sns.distplot(Asw_s_max)
plt.axvline(x=np.mean(Asw_s_max),c='red',ls='--',label='mean')
plt.axvline(x=np.percentile(Asw_s_max,25),c='green',ls='--',label='25-75th percentile')
plt.axvline(x=np.percentile(Asw_s_max,75),c='green',ls='--')
plt.legend()

In [ ]:
a=Rebar.groupby(["ID"]).max()
a.plot.scatter(x='Length', y='p_tol');

In [ ]:
#Rebar.loc[Rebar.p_tol>0.021]
#a = ratio_max[ratio_max<0.004]
#a.to_csv('Output_CheckBeam.csv')
#a.to_csv('ratio_max<0.004.csv')
#Asw_s_max[Asw_s_max>2]

Import Rebar Table - can manually change rebar arrangement

In [ ]:
df_rebar = pd.read_excel('Rebar Table.xlsm',sheet_name=0)
df_rebar.dropna(axis=0, inplace=True, how='all')
df_rebar=df_rebar.fillna(0)
df_rebar=df_rebar.astype("string")
df_rebar
for i in range(df_rebar.shape[1]):
    col_name = df_rebar.columns[i]
    if i==0 or i%2==0:
        df_rebar[col_name] = df_rebar[col_name].apply(lambda x: float(x))

df_linkbar = pd.read_excel('Rebar Table.xlsm',sheet_name=1)
df_linkbar.dropna(axis=0, inplace=True, how='all')
df_linkbar=df_linkbar.fillna(0)
df_linkbar=df_linkbar.astype("string")
for i in range(df_linkbar.shape[1]):
    col_name = df_linkbar.columns[i]
    if i==0 or i%2==0:
        df_linkbar[col_name] = df_linkbar[col_name].apply(lambda x: float(x))

In [ ]:
def mainbar(b: float, as_req: float, df_rebar: pd.DataFrame())->str:
    b = int(float(b))
    try:
        as_req_name = "{0}_rebar".format(b)
        #print(as_req_name)
        query_f = '@df_rebar[{0}]>={1}'.format(b, as_req)
        rebars = df_rebar.query(query_f)[as_req_name] 
        as_rebars = df_rebar.query(query_f)[b] 
        selected_bar = rebars.iloc[0]
        selected_as = as_rebars.iloc[0]
        return selected_bar,selected_as
    except Exception as error:
        print('Caught this error: ' + repr(error), b, as_req)


def linkbar(b: float, Asw_s: float, df_linkbar: pd.DataFrame())->str:
    b = int(float(b))
    try:
        Asw_s_name = "{0}_linkbar".format(b)
        #print(Asw_s_name)
        query_f = '@df_linkbar[{0}]>={1}'.format(b, Asw_s)
        linkbars = df_linkbar.query(query_f)[Asw_s_name]
        Asw_s_linkbars = df_linkbar.query(query_f)[b] 
        selected_linkbar = linkbars.iloc[0]
        selected_Asw_s = Asw_s_linkbars.iloc[0]
        return selected_linkbar,selected_Asw_s
    except Exception as error:
        print('Caught this error: ' + repr(error), b, Asw_s)


def ms(T):
    ppi = 3.1415926
    rebar_list = T.split(' / ')
    total_area = 0
    for bar in rebar_list:
        d = int(bar[bar.index('T')+1:bar.index('T')+3])
        n = int(bar[:bar.index('T')])
        x=[]
        x.append(d)
        print(x)
        area = n * ppi * d ** 2 / 4
        total_area += area
    return total_area


def rebar(arrangement):
    layers = arrangement.split(" / ")
    max_diameter = 0
    num_layers = len(layers)

    for layer in layers:
        match = re.match(r'(\d+)T(\d+)', layer)
        if not match:
            return "Invalid input format"
        diameter = int(match.group(2))
        if diameter > max_diameter:
            max_diameter = diameter

    return max_diameter,num_layers


def calculate_d_ef(rebar, h, a_cover):
    dia = max(rebar[0], 25)
    layers = rebar[1]
    return h - a_cover - 12 - (2 * layers - 1) * dia * 0.5

In [ ]:
top_rebar = Rebar.apply(lambda row: mainbar(row['B'], row['As_top'],df_rebar), axis=1)
bot_rebar = Rebar.apply(lambda row: mainbar(row['B'], row['As_bot'],df_rebar), axis=1)
linkbar = Rebar.apply(lambda row: linkbar(row['B'], row["Asw_s"],df_linkbar), axis=1)

In [ ]:
top_selected_rebar = [result[0] for result in top_rebar]
top_as_selected = [result[1] for result in top_rebar]

Rebar['Top_Rebar'] = top_selected_rebar
Rebar['Top_As_Rebar'] = top_as_selected

bot_selected_rebar = [result[0] for result in bot_rebar]
bot_as_selected = [result[1] for result in bot_rebar]

Rebar['Bot_Rebar'] = bot_selected_rebar
Rebar['Bot_As_Rebar'] = bot_as_selected

link_selected = [result[0] for result in linkbar]
link_Asw_selected = [result[1] for result in linkbar]

Rebar['Linkbar'] = link_selected
Rebar['Asw_Linkbar'] = link_Asw_selected


selected_rebar=Rebar[["Span type",'B','D',"As_top","As_bot","p_top","p_bot","Asw_s",
                    'Top_Rebar','Top_As_Rebar','Bot_Rebar','Bot_As_Rebar','Linkbar','Asw_Linkbar','Length']]
selected_rebar["Top uti"]=selected_rebar.As_top/selected_rebar.Top_As_Rebar
selected_rebar["Bot uti"]=selected_rebar.As_bot/selected_rebar.Bot_As_Rebar
selected_rebar["Asw uti"]=selected_rebar.Asw_s/selected_rebar.Asw_Linkbar
selected_rebar.to_csv('Select Rebar.csv')

Re-calculate effective Depth d_ef

In [109]:
selected_rebar = pd.read_csv("Select Rebar.csv")
selected_rebar['d_ef_top'] = selected_rebar.apply(lambda row: calculate_d_ef(rebar(row['Top_Rebar']), row['D'], a_cover), axis=1)
selected_rebar['d_ef_bot'] = selected_rebar.apply(lambda row: calculate_d_ef(rebar(row['Bot_Rebar']), row['D'], a_cover), axis=1)


selected_rebar_filtered = selected_rebar[(selected_rebar['d_ef_top'] < selected_rebar['D']-70) | (selected_rebar['d_ef_bot'] < selected_rebar['D']-70)]
selected_rebar_filtered['d_ef'] = selected_rebar_filtered[['d_ef_top', 'd_ef_bot']].min(axis=1)

min_d_ef = selected_rebar_filtered.groupby("Name")["d_ef"].min()
name_filtered = pd.unique(selected_rebar_filtered['Name']).tolist()

filtered_df = Beam[Beam['Name'].isin(name_filtered)]
filtered_df = filtered_df.merge(min_d_ef, on="Name", suffixes=('', '_min_d_ef'))

As_cal_1=[As_rq(fck,fyk,M,D,B) for M,D,B in zip(filtered_df.Myy,filtered_df.d_ef+70,filtered_df.B)]
Result_1=filtered_df.loc[:,["Name","Span type","ID","seg",'Length','B','D',"Myy",'Fz']]
Result_1["As"], Result_1["Asc"], Result_1["As_top"], Result_1["As_bot"],Result_1["k"],Result_1["z"]=  zip(*As_cal_1)

Rebar_1=Result_1[["Name","Span type","ID","seg","Length",'B','D',"As_top","As_bot"]].groupby(["Name","ID","seg"]).max()
Rebar_1["As_tol"]=Rebar_1.As_top + Rebar_1.As_bot
Rebar_1["p_top"]=Rebar_1.As_top/Rebar_1.B/Rebar_1.D
Rebar_1["p_bot"]=Rebar_1.As_bot/Rebar_1.B/Rebar_1.D
Rebar_1["p_tol"]=Rebar_1.As_tol/Rebar_1.B/Rebar_1.D
Rebar_1["Bend_status"]=Rebar_1.p_tol.apply(lambda x: 0 if x <= 0.04 else 1)
Rebar_1=Rebar_1[["Span type","Length",'B','D',"As_top","As_bot","p_top","p_bot","p_tol","Bend_status"]]
Rebar_1.loc[(Rebar_1.Bend_status == 1)]

C:\Users\HUAN~1.TRA\AppData\Local\Temp/ipykernel_5688/817339409.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rebar_filtered['d_ef'] = selected_rebar_filtered[['d_ef_top', 'd_ef_bot']].min(axis=1)


,,,Span type,Length,B,D,As_top,As_bot,p_top,p_bot,p_tol,Bend_status
Name,ID,seg,,,,,,,,,,


In [ ]:
top_rebar_1 = Rebar_1.apply(lambda row: mainbar(row['B'], row['As_top'],df_rebar), axis=1) #choosing new rebar by new As
bot_rebar_1 = Rebar_1.apply(lambda row: mainbar(row['B'], row['As_bot'],df_rebar), axis=1) #choosing new rebar by new As

In [110]:
top_selected_rebar_1 = [result[0] for result in top_rebar_1]
top_as_selected_1 = [result[1] for result in top_rebar_1]

Rebar_1['Top_Rebar'] = top_selected_rebar_1
Rebar_1['Top_As_Rebar'] = top_as_selected_1

bot_selected_rebar_1 = [result[0] for result in bot_rebar_1]
bot_as_selected_1 = [result[1] for result in bot_rebar_1]

Rebar_1['Bot_Rebar'] = bot_selected_rebar_1
Rebar_1['Bot_As_Rebar'] = bot_as_selected_1
Rebar_1 = Rebar_1.reset_index()


columns_to_update = ["As_top","As_bot","p_top","p_bot",'Top_Rebar','Top_As_Rebar','Bot_Rebar','Bot_As_Rebar']
mappings = {col: Rebar_1.set_index(['Name', 'seg'])[col].to_dict() for col in columns_to_update}
column_order = ["Name", "ID", "seg", "Span type", "B", "D", "As_top", "As_bot", "p_top", "p_bot",'Asw_s','Top_Rebar', 'Top_As_Rebar',
                 'Bot_Rebar', 'Bot_As_Rebar', 'Linkbar','Asw_Linkbar','Length']

selected_rebar_update = pd.DataFrame(columns=column_order)
for col in column_order:
    if col in columns_to_update:
        selected_rebar_update[col] = selected_rebar.set_index(['Name', 'seg']).index.map(mappings[col])
        selected_rebar_update[col] = selected_rebar_update[col].fillna(selected_rebar[col])
    else:
        selected_rebar_update[col] = selected_rebar[col]

selected_rebar_update["Top uti"]=selected_rebar_update.As_top/selected_rebar_update.Top_As_Rebar
selected_rebar_update["Bot uti"]=selected_rebar_update.As_bot/selected_rebar_update.Bot_As_Rebar
selected_rebar_update["Asw uti"]=selected_rebar_update.Asw_s/selected_rebar_update.Asw_Linkbar
selected_rebar_update.to_csv('Select Rebar Re-cal.csv')

# Merge Rebar for Beams Length <4m 1st

In [113]:
selected_rebar=pd.read_csv("Select Rebar Re-cal.csv")
selected_rebar_update =  selected_rebar.copy(deep = False)
grouped_lists = np.array_split(selected_rebar_update.values, len(selected_rebar) // 3)
# print(grouped_lists)
for i in range(0,beam_number):
    length = grouped_lists[i][0][17]
    if length < 4:
        temp_as_top = [grouped_lists[i][j][12] for j in range(0,3)]
        max_as_top = max(temp_as_top)
        temp_rebar_top = [grouped_lists[i][j][11] for j in range(0,3)]
        max_rebar_top = temp_rebar_top[temp_as_top.index(max_as_top)]
        for k in range(i*3, i*3+3):
            selected_rebar_update.loc[k,"Top_As_Rebar"]=max_as_top
            selected_rebar_update.loc[k,"Top_Rebar"]=max_rebar_top
            
            
        temp_as_bot = [grouped_lists[i][j][14] for j in range(0,3)]
        max_as_bot = max(temp_as_bot)
        temp_rebar_bot = [grouped_lists[i][j][13] for j in range(0,3)]
        max_rebar_bot = temp_rebar_bot[temp_as_bot.index(max_as_bot)]
        for h in range(i*3, i*3+3):
            selected_rebar_update.loc[h,"Bot_As_Rebar"]=max_as_bot
            selected_rebar_update.loc[h,"Bot_Rebar"]=max_rebar_bot

#selected_rebar_update.to_csv('Select Rebar Update 0.csv')

# Merge Rebar for Beams have same beam width 1st

In [ ]:
grouped_lists = np.array_split(selected_rebar_update.values, len(selected_rebar) // 3)
for i in range(0,beam_number-1):
    span_type = grouped_lists[i][0][3]
    b_current = grouped_lists[i][0][4]
    b_next = grouped_lists[i+1][0][4]
    if b_current == b_next:
        if span_type in ["1st","int"]:
            top_current_as=grouped_lists[i][2][12]
            top_current_bar=grouped_lists[i][2][11]
            top_next_as=grouped_lists[i+1][0][12]
            top_next_bar=grouped_lists[i+1][0][11]

            if top_current_as > top_next_as:
                new_top_as = top_current_as
                new_top_bar =  top_current_bar
                index_top = i*3+3
            else:
                new_top_as = top_next_as
                new_top_bar = top_next_bar
                index_top = i*3+2               
                
                
            
            bot_current_as=grouped_lists[i][2][14]
            bot_current_bar=grouped_lists[i][2][13]
            bot_next_as=grouped_lists[i+1][0][14]
            bot_next_bar=grouped_lists[i+1][0][13]

            if bot_current_as > bot_next_as:
                new_bot_as = bot_current_as
                new_bot_bar =  bot_current_bar
                index_bot = i*3+3
            else:
                new_bot_as = bot_next_as
                new_bot_bar = bot_next_bar
                index_bot = i*3+2      
            selected_rebar_update.loc[index_top,"Top_As_Rebar"]=new_top_as
            selected_rebar_update.loc[index_top,"Top_Rebar"]=new_top_bar
            selected_rebar_update.loc[index_bot,"Bot_As_Rebar"]=new_bot_as
            selected_rebar_update.loc[index_bot,"Bot_Rebar"]=new_bot_bar
            
#selected_rebar_update.to_csv('Select Rebar Update 1.csv')

# Merge Rebar for beams have length <4m 2nd

In [ ]:
grouped_lists = np.array_split(selected_rebar_update.values, len(selected_rebar) // 3)
for i in range(0,beam_number):
    length = grouped_lists[i][0][17]
    if length <4:
        temp_as_top = [grouped_lists[i][j][12] for j in range(0,3)]
        max_as_top = max(temp_as_top)
        temp_rebar_top = [grouped_lists[i][j][11] for j in range(0,3)]
        max_rebar_top = temp_rebar_top[temp_as_top.index(max_as_top)]
        for k in range(i*3, i*3+3):
            selected_rebar_update.loc[k,"Top_As_Rebar"]=max_as_top
            selected_rebar_update.loc[k,"Top_Rebar"]=max_rebar_top
            
            
        temp_as_bot = [grouped_lists[i][j][14] for j in range(0,3)]
        max_as_bot = max(temp_as_bot)
        temp_rebar_bot = [grouped_lists[i][j][13] for j in range(0,3)]
        max_rebar_bot = temp_rebar_bot[temp_as_bot.index(max_as_bot)]
        for h in range(i*3, i*3+3):
            selected_rebar_update.loc[h,"Bot_As_Rebar"]=max_as_bot
            selected_rebar_update.loc[h,"Bot_Rebar"]=max_rebar_bot

#selected_rebar_update.to_csv('Select Rebar Update 2.csv')

# Merge Rebar for Beams have same beam width 2nd

In [ ]:
grouped_lists = np.array_split(selected_rebar_update.values, len(selected_rebar) // 3)
for i in range(0,beam_number-1):
    span_type = grouped_lists[i][0][3]
    b_current = grouped_lists[i][0][4]
    b_next = grouped_lists[i+1][0][4]
    if b_current == b_next:
        if span_type in ["1st","int"]:
            top_current_as=grouped_lists[i][2][12]
            top_current_bar=grouped_lists[i][2][11]
            top_next_as=grouped_lists[i+1][0][12]
            top_next_bar=grouped_lists[i+1][0][11]

            if top_current_as > top_next_as:
                new_top_as = top_current_as
                new_top_bar =  top_current_bar
                index_top = i*3+3
            else:
                new_top_as = top_next_as
                new_top_bar = top_next_bar
                index_top = i*3+2               
                
                
            
            bot_current_as=grouped_lists[i][2][14]
            bot_current_bar=grouped_lists[i][2][13]
            bot_next_as=grouped_lists[i+1][0][14]
            bot_next_bar=grouped_lists[i+1][0][13]

            if bot_current_as > bot_next_as:
                new_bot_as = bot_current_as
                new_bot_bar =  bot_current_bar
                index_bot = i*3+3
            else:
                new_bot_as = bot_next_as
                new_bot_bar = bot_next_bar
                index_bot = i*3+2      
            selected_rebar_update.loc[index_top,"Top_As_Rebar"]=new_top_as
            selected_rebar_update.loc[index_top,"Top_Rebar"]=new_top_bar
            selected_rebar_update.loc[index_bot,"Bot_As_Rebar"]=new_bot_as
            selected_rebar_update.loc[index_bot,"Bot_Rebar"]=new_bot_bar
            
#selected_rebar_update.to_csv('Select Rebar Update 3.csv')

# Merge Rebar for Beams have different beam width

In [ ]:
grouped_lists = np.array_split(selected_rebar_update.values, len(selected_rebar) // 3)
for i in range(0,beam_number-1):
    span_type = grouped_lists[i][0][3]
    b_current = grouped_lists[i][0][4]
    b_next = grouped_lists[i+1][0][4]
    if b_current != b_next:
        if span_type in ["1st","int"]:
            top_current_as=grouped_lists[i][2][12]
            top_current_bar=grouped_lists[i][2][11]
            top_next_as=grouped_lists[i+1][0][12]
            top_next_bar=grouped_lists[i+1][0][11]
            top_current_layer = len(top_current_bar.split('/'))
            top_next_layer = len(top_next_bar.split('/'))  
            max_no_layer = max(top_current_layer,top_next_layer)

            if max_no_layer < 2:
                top_current_dia = top_current_bar.split('T')[1]
                top_next_dia = top_next_bar.split('T')[1]
                max_dia = max(top_current_dia , top_next_dia)
                top_current_new = top_current_bar.split('T')[0] + 'T' + max_dia
                top_next_new = top_next_bar.split('T')[0]+ 'T' + max_dia
                selected_rebar_update.loc[i*3+2,"Top_Rebar"]=top_current_new
                selected_rebar_update.loc[i*3+3,"Top_Rebar"]=top_next_new
                selected_rebar_update.loc[i*3+2,"Top_As_Rebar"]=ms(top_current_new)
                selected_rebar_update.loc[i*3+3,"Top_As_Rebar"]=ms(top_next_new)


            if max_no_layer >1:
                cur_dia = [int(top_current_bar.split('/')[n].split('T')[1]) for n in range(0,top_current_layer)]
                cur_n = [top_current_bar.split('/')[n].split('T')[0] for n in range(0,top_current_layer)]
                next_dia = [int(top_next_bar.split('/')[n].split('T')[1]) for n in range(0,top_next_layer)]
                next_n = [top_next_bar.split('/')[n].split('T')[0] for n in range(0,top_next_layer)]
                max_dia = []
                for x, y in zip_longest(cur_dia, next_dia, fillvalue=float('-inf')):
                    max_dia.append(max(x, y))
                top_current_new = ' /'.join([cur_n[n] + 'T' + str(max_dia[n]) for n in range(0,top_current_layer)])
                top_next_new = ' /'.join([next_n[n] + 'T' + str(max_dia[n]) for n in range(0,top_next_layer)])
                selected_rebar_update.loc[i*3+2,"Top_Rebar"]=top_current_new
                selected_rebar_update.loc[i*3+3,"Top_Rebar"]=top_next_new
                selected_rebar_update.loc[i*3+2,"Top_As_Rebar"]=ms(top_current_new)
                selected_rebar_update.loc[i*3+3,"Top_As_Rebar"]=ms(top_next_new)

                                    
                
            bot_current_as=grouped_lists[i][2][14]
            bot_current_bar=grouped_lists[i][2][13]
            bot_next_as=grouped_lists[i+1][0][14]
            bot_next_bar=grouped_lists[i+1][0][13]
            bot_current_layer = len(bot_current_bar.split('/'))
            bot_next_layer = len(bot_next_bar.split('/'))
            bot_current_dia = bot_current_bar.split('T')[1]
            bot_next_dia = bot_next_bar.split('T')[1]
            if max(bot_current_layer,bot_next_layer) < 2:
                max_dia = max(bot_current_dia , bot_next_dia)
                bot_current_new = bot_current_bar.split('T')[0] + 'T' + max_dia
                bot_next_new = bot_next_bar.split('T')[0]+ 'T' + max_dia
                selected_rebar_update.loc[i*3+2,"Bot_Rebar"]=bot_current_new
                selected_rebar_update.loc[i*3+3,"Bot_Rebar"]=bot_next_new
                selected_rebar_update.loc[i*3+2,"Bot_As_Rebar"]=ms(bot_current_new)
                selected_rebar_update.loc[i*3+3,"Bot_As_Rebar"]=ms(bot_next_new)
            if max_no_layer >1:
                cur_dia = [int(bot_current_bar.split('/')[n].split('T')[1]) for n in range(0,bot_current_layer)]
                cur_n = [bot_current_bar.split('/')[n].split('T')[0] for n in range(0,bot_current_layer)]
                next_dia = [int(bot_next_bar.split('/')[n].split('T')[1]) for n in range(0,bot_next_layer)]
                next_n = [bot_next_bar.split('/')[n].split('T')[0] for n in range(0,bot_next_layer)]
                max_dia = []
                for x, y in zip_longest(cur_dia, next_dia, fillvalue=float('-inf')):
                    max_dia.append(max(x, y))
                bot_current_new = ' /'.join([cur_n[n] + 'T' + str(max_dia[n]) for n in range(0,bot_current_layer)])
                bot_next_new = ' /'.join([next_n[n] + 'T' + str(max_dia[n]) for n in range(0,bot_next_layer)])
                selected_rebar_update.loc[i*3+2,"Bot_Rebar"]=bot_current_new
                selected_rebar_update.loc[i*3+3,"Bot_Rebar"]=bot_next_new
                selected_rebar_update.loc[i*3+2,"Bot_As_Rebar"]=ms(bot_current_new)
                selected_rebar_update.loc[i*3+3,"Bot_As_Rebar"]=ms(bot_next_new)

#selected_rebar_update.to_csv('Select Rebar Update.csv')

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

selected_rebar = pd.read_csv("Select Rebar.csv")

diff = selected_rebar.loc[:, 'Top_As_Rebar'] != selected_rebar_update.loc[:, 'Top_As_Rebar']
diff2 = selected_rebar.loc[:, 'Bot_As_Rebar'] != selected_rebar_update.loc[:, 'Bot_As_Rebar']
selected_rebar_update['Top update?'] = diff
selected_rebar_update['Bot update?'] = diff2

df = selected_rebar_update
wb = Workbook()
ws = wb.active


for col_num, column_title in enumerate(df.columns, 1):
    ws.cell(row=1, column=col_num, value=column_title)

start_row = 2
num_rows = 3
num_repeats = int(len(df.index)/3)

highlight_color = 'FFC7CE'
unhighlight_color = 'FFFFFF'


for index, row in df.iterrows():
    row_num = index + start_row
    for col_num, value in enumerate(row, 1):
        ws.cell(row=row_num, column=col_num, value=value)

    if index % (num_rows * 2) < num_rows:
        for col_num in range(1, df.shape[1]+1):
            ws.cell(row=row_num, column=col_num).fill = PatternFill(start_color=highlight_color, end_color=highlight_color, fill_type='solid')
    else:
        for col_num in range(1, df.shape[1]+1):
            ws.cell(row=row_num, column=col_num).fill = PatternFill(start_color=unhighlight_color, end_color=unhighlight_color, fill_type='solid')

wb.save('Select Rebar Update.xlsx')



In [ ]:
df_update_rebar = pd.read_excel('Select Rebar Update.xlsx')
new_top_rebar = df_update_rebar.loc[:,'Top_Rebar']
new_bot_rebar = df_update_rebar.loc[:,'Bot_Rebar']
df_update_rebar['Top_As_Rebar'] = new_top_rebar.apply(ms)
df_update_rebar['Bot_As_Rebar'] = new_bot_rebar.apply(ms)
df_update_rebar["Top uti"]=df_update_rebar.As_top/df_update_rebar.Top_As_Rebar
df_update_rebar["Bot uti"]=df_update_rebar.As_bot/df_update_rebar.Bot_As_Rebar
df_update_rebar["Asw uti"]=df_update_rebar.Asw_s/df_update_rebar.Asw_Linkbar

df = df_update_rebar
wb = Workbook()
ws = wb.active


for col_num, column_title in enumerate(df.columns, 1):
    ws.cell(row=1, column=col_num, value=column_title)

start_row = 2
num_rows = 3
num_repeats = int(len(df.index)/3)

highlight_color = 'FFC7CE'
unhighlight_color = 'FFFFFF'


for index, row in df.iterrows():
    row_num = index + start_row
    for col_num, value in enumerate(row, 1):
        ws.cell(row=row_num, column=col_num, value=value)

    if index % (num_rows * 2) < num_rows:
        for col_num in range(1, df.shape[1]+1):
            ws.cell(row=row_num, column=col_num).fill = PatternFill(start_color=highlight_color, end_color=highlight_color, fill_type='solid')
    else:
        for col_num in range(1, df.shape[1]+1):
            ws.cell(row=row_num, column=col_num).fill = PatternFill(start_color=unhighlight_color, end_color=unhighlight_color, fill_type='solid')


wb.save('Select Rebar Update Final.xlsx')
